# 원본 데이터 병합, 전처리

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 웰니스 대화 스크립트 데이터셋(train/valid set)
# https://aihub.or.kr/aihubdata/data/view.do?currMenu=120&topMenu=100&aihubDataSe=extrldata&dataSetSn=267
df1 = pd.read_excel('./data/감성대화말뭉치(최종데이터)_Training.xlsx')[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]
df2 = pd.read_excel('./data/감성대화말뭉치(최종데이터)_Validation.xlsx')[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]

# 노년층 대상 감성 분류 모델
# https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&dataSetSn=453&aihubDataSe=extrldata
df3 = pd.read_csv('./data/train.csv')[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]

# 모든 데이터 결합
df = pd.concat([df1,df2,df3])

# 널값 ''로 채우기
df = df.fillna('')

# 사람문장1, 사람문장2, 사람문장3 전부 더해서 'Q' 컬럼 생성
df['Q'] = df.apply(lambda x : x['사람문장1'] + ' ' + x['사람문장2'] + ' ' + x['사람문장3'], axis = 1)

# 필요 컬럼 생성, 리네임
df = df[['감정_대분류','Q']]
df.columns = ['emotion', 'Q']

# 당황 제외
df = df[df['emotion'] != '당황']

# 라벨 인코더 활용 emotion 라벨화
le = LabelEncoder()
df['emotion_label'] = le.fit_transform(df['emotion'])

# 데이터프레임 인덱스 리셋, 중복 제거
df.reset_index(drop = True, inplace = True)
df = df.drop_duplicates()

# 저장
df.to_csv('./data/감성대화최종.csv', index = False)

# 챗봇 데이터 증폭기

In [ ]:
import pandas as pd
import random

df = pd.read_csv('C:/PlayData/Emotional_DJ_chatbot/data/wellness_dataset_original_LAST.csv', encoding = 'cp949')

# 챗봇 데이터 증폭기
# 빈 데이터를 같은 구분 값의 랜덤 값으로 넣어줌

next_df = df.loc[df['구분'] == '증상/불면', '챗봇']
answer_list = list(next_df[next_df.isna() == False])
for idx in next_df[next_df.isna() == True].index:
    df.loc[idx, '챗봇']= answer_list[random.randrange(0,len(answer_list))]
    
df.columns = ['구분', 'Q', 'A']
df.to_csv('./data/WellNese_Preprocess_LAST.csv', index = False)

# json 추출 

In [ ]:
import pandas as pd
import json

# json 파일 접근
with open('./data/감성대화말뭉치(최종데이터)_Training.json') as f:
    js = json.loads(f.read())

# 감정분류
def emotion_clf(code):
    if code == 1:
        return '분노'
    elif code == 2:
        return '슬픔'
    elif code == 3:
        return '불안'
    elif code == 4:
        return '상처'
    elif code == 5:
        return '기쁨'
    elif code == 6:
        return '당황'


# 원하는 값 Q, emotion 리스트에 저장
Q = []
emotion = []
for i in range(len(js)):
    Q.append(js[i]['talk']['content']['HS01'])
    emotion.append(int(js[i]['profile']['emotion']['emotion-id'][-2:-1]))

# Q, emotion 리스트로 데이터프레임 만들기
new_data = pd.DataFrame(data = list(zip(Q , emotion)), columns = ['Q', 'emotion'])

# 감정 분류
new_data['emotion_kor'] = new_data['emotion'].apply(emotion_clf)
new_data

# 저장
new_data.to_csv('감정분류텍스트.csv', index = False)